# CaLPA Scratch Folder

This is a scratch notebook used to test the code and functionality of the AI California Legislative Policy Analysis (CALPA) system. It is not intended for production use and may contain incomplete or experimental code. The purpose of this notebook is to facilitate the development and testing of the CALPA system, including its data processing, analysis, and visualization components. The notebook may include code snippets, comments, and notes related to the development process. Please refer to the official documentation and user guides for the CALPA system for more information on its usage and features.

In [ ]:
#%reset

## Initialization

In [1]:
# Import required libraries
import os
import time
from datetime import date
from datetime import datetime
import json
import mimetypes
import glob
import base64
import zipfile
import io
import dotenv
import requests
import pandas as pd
import feedparser
import webbrowser
from mrkdwn_analysis import MarkdownAnalyzer

# Import the calpa library
from calpa import calpa

In [2]:
# Load environment variables from .env file
dotenv.load_dotenv(os.path.join(os.getcwd(), '.env'))

# Create project metadata for the AI project
prjMetadata = calpa.projectMetadata(1)

# Create the project directories dictionary
prjDirs = calpa.projectDirectories(os.getcwd())

# Instantiate the LegiScan and Calpa classes
legiscan = calpa.LegiScan()


Project Global Settings:
- Name: CaLPA
- Title: AI Legislative Policy Analysis
- Description: California Legislative Policy Analysis for Artificial Intelligence Related Bills
- Project Step: Part 1 - Preliminary Operations
- Version: 1.0
- Author: Dr. Kostas Alexandridis, GISP
- Periods: 2009-2010, 2011-2012, 2013-2014, 2015-2016, 2017-2018, 2019-2020, 2021-2022, 2023-2024, 2025-2026
- Start Date: 2010-12-02
- End Date: 2025-04-27
Directory Global Settings:

General:
- Project (pathPrj): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA
- Admin (pathAdmin): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\admin
- Metadata (pathMetadata): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\metadata
- Analysis (pathAnalysis): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\analysis
- Obsidian Vault (pathObsidian): C:\Users\ktale\Knowledge Management\Policy and Governance\Legislation
Scripts:
- Python Calpa Module (pathScriptsCalpa): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\calpa
- Markdown Sc

In [3]:
# Codebook lookup variables
codebookLookupVars = [var for var in dir(calpa.codebook) if var.startswith('lookup')]
codebookDictVars = [var for var in dir(calpa.codebook) if var.startswith('dict')]
print(f"Codebook Lookup Variables:\n{codebookLookupVars}\n")
print(f"Codebook Dictionary Variables:\n{codebookDictVars}\n")

Codebook Lookup Variables:
['lookupBillCode', 'lookupBillTextType', 'lookupBillType', 'lookupBodyType', 'lookupEventType', 'lookupMimeType', 'lookupPartyType', 'lookupProgressType', 'lookupReasonType', 'lookupRoleType', 'lookupSastType', 'lookupSponsorType', 'lookupStateType', 'lookupStatusType', 'lookupSupplementType', 'lookupVoteType']

Codebook Dictionary Variables:
['dictGetAmendment', 'dictGetBill', 'dictGetBillText', 'dictGetPerson', 'dictGetRollCall', 'dictGetSessionList', 'dictGetSupplement']



In [4]:
# Obtain the stored sessions list from JSON dictionary on disk (data/lookup directory)
sessionListStored = legiscan.getStoredData(dataType = "session")

# Obtain the stored session People list from JSON dictionary on disk (data/lookup directory)
sessionPeopleStored = legiscan.getStoredData(dataType = "people")

# Obtain the stored dataset list from JSON dictionary on disk (data/lookup directory)
datasetListStored = legiscan.getStoredData(dataType = "dataset")

# Get the stored raw master list from JSON dictionary on disk (data/lookup directory)
masterListRawStored = legiscan.getStoredData(dataType = "master", raw = True)
# Get the stored master list from JSON dictionary on disk (data/lookup directory)
masterListStored = legiscan.getStoredData(dataType = "master", raw = False)

# Get the AI monitoring list from disk (data/lookup directory)
aiBillListStored = legiscan.getStoredData(dataType = "bills", project = "AI")

# Get the AI full list of bills from dism (data/legis/json directory)
aiBills = legiscan.getStoredData(dataType = "data", project = "AI")

# Get the AI bill summries list from disk (data/lookup directory)
aiBillsSummariesStored = legiscan.getStoredData(dataType = "summaries", project = "AI")

## End of Initialization

In [ ]:
# open test["main"] in a web browser
webbrowser.open(test["status"])

In [ ]:
webbrowser.open(aiBills["2013-2014"]["AB1465"]["url"], new=2, autoraise=True)

In [ ]:
webbrowser.open(aiBills["2013-2014"]["AB1465"]["state_link"], new=2, autoraise=True)

In [ ]:
legiscan.summarizeBillSponsors(aiBills["2013-2014"]["SB836"], output="md")

- Primary Sponsor(s): [Ellen Corbett (D, SD10)](https://ballotpedia.org/Ellen_Corbett)
- Co-Sponsor(s): [Joel Anderson (R, SD38)](https://ballotpedia.org/Joel_Anderson_(California)), [Jim Beall (D, SD15)](https://ballotpedia.org/James_Beall_Jr.), [Marty Block (D, SD39)](https://ballotpedia.org/Martin_Block), [Anthony Cannella (R, SD12)](https://ballotpedia.org/Anthony_Cannella), [Lou Correa (D, SD34)](https://ballotpedia.org/Lou_Correa), [Noreen Evans (D, SD02)](https://ballotpedia.org/Noreen_Evans), [Cathleen Galgiani (D, SD05)](https://ballotpedia.org/Cathleen_Galgiani), [Loni Hancock (D, SD09)](https://ballotpedia.org/Loni_Hancock), [Jerry Hill (D, SD13)](https://ballotpedia.org/Gerald_Hill), [Ricardo Lara (D, SD33)](https://ballotpedia.org/Ricardo_Lara), [Ted Lieu (D, SD28)](https://ballotpedia.org/Ted_Lieu), [Carol Liu (D, SD25)](https://ballotpedia.org/Carol_Liu), [Jim Nielsen (R, SD04)](https://ballotpedia.org/Jim_Nielsen), [Richard Roth (D, SD31)](https://ballotpedia.org/Richard_Roth), [Lois Wolk (D, SD03)](https://ballotpedia.org/Lois_Wolk)
- Joint Sponsor(s): [Nancy Skinner (D, SD09)](https://ballotpedia.org/Nancy_Skinner_(California)), [Marie Waldron (R, AD75)](https://ballotpedia.org/Marie_Waldron)

In [ ]:

analyzer = MarkdownAnalyzer(samplePath)

headers = analyzer.identify_headers()
paragraphs = analyzer.identify_paragraphs()
blockquotes = analyzer.identify_blockquotes()
links = analyzer.identify_links()
codeBlocks = analyzer.identify_code_blocks()

In [ ]:
analysis = analyzer.analyse()

print(analysis)

In [ ]:
headers

In [ ]:
paragraphs

In [ ]:
links

In [ ]:
import io
import pypandoc
import panflute

def action(elem, doc):
    if isinstance(elem, panflute.Image):
        doc.images.append(elem)
    elif isinstance(elem, panflute.Link):
        doc.links.append(elem)

if __name__ == '__main__':
    data = pypandoc.convert_file('example.md', 'json')
    doc = panflute.load(io.StringIO(data))
    doc.images = []
    doc.links = []
    doc = panflute.run_filter(action, prepare=prepare, doc=doc)

    print("\nList of image URLs:")
    for image in doc.images:
        print(image.url)

In [6]:
for key, value in aiBills.items():
    print(f"{key} Legislative Session:")
    for billId, bill in value.items():
        legiscan.aiBillMarkdown(
            billPeriod = key,
            billId = billId,
            billsDict = aiBills,
            billsSummariesDict = aiBillsSummariesStored,
            obsidianSync = True
        )
        print(f"- {billId}: {bill['title']}")

2013-2014 Legislative Session:
- AB1465: Education finance: education omnibus trailer bill.
- SB836: Brain research: Cal-BRAIN program.
- SB860: Education finance: education omnibus trailer bill.
2017-2018 Legislative Session:
- AB1809: Higher education trailer bill.
- AB2662: State government: Office of Planning and Research: electronic media: child development.
- ACR215: 23 Asilomar AI Principles.
- SB843: Higher education trailer bill.
- SB1470: Commission on the Future of Work.
2019-2020 Legislative Session:
- AB156: Optometrists: endorsement fee.
- AB459: State government: Artificial Intelligence in State Government Services Commission: report.
- AB485: Local government: economic development subsidies.
- AB594: Artificial intelligence.
- AB976: Artificial Intelligence in State Government Services Commission.
- AB1576: Secretary of Government Operations: working group: technology.
- AB2269: Personal rights: automated decision systems.
- AB3317: Unfair and deceptive practices.
- AB3